In [ ]:
import matplotlib.pyplot as plt
from datasets.PupilCoreDatasetIfOpened import PupilCoreDatasetIfOpened
from datasets.PupilCoreDatasetPupil import PupilCoreDatasetPupil
from datasets.PupilCoreDatasetCornealReflection import PupilCoreDatasetCornealReflection
from models.ifOpened import ifOpenedModel
from models.pupilDetectModel import PupilDetectModel
from models.utils import train_first_model
import models.utils as utils
import torch
from torch import nn
from torch.optim import lr_scheduler
import torch.optim as optim
import time
import copy
from torchvision import models
import numpy as np
import cv2


In [ ]:
# dataset = PupilCoreDatasetIfOpened(
#     "datasets/PupilCoreDataset/video5_eye0_video.avi",
#     'datasets/PupilCoreDataset/video5_eye0_pupildata.csv',
#     "datasets/PupilCoreDataset/video5_eye1_video.avi",
#     'datasets/PupilCoreDataset/video5_eye1_pupildata.csv'
# )

In [ ]:
# indices = torch.randperm(len(dataset)).tolist()
# train_part = int(0.8 * len(dataset))
# train_dataset = torch.utils.data.Subset(dataset, indices[:train_part])
# test_dataset = torch.utils.data.Subset(dataset, indices[train_part:])
# dataset_sizes = {
#     'train': len(train_dataset),
#     'test': len(test_dataset)
# }

# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)
# dataloaders = {
#     "train": train_dataloader,
#     "test": test_dataloader
# }

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# model = ifOpenedModel()
# model.to(device)

In [ ]:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# model = train_first_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=2, dataloaders=dataloaders, dataset_sizes=dataset_sizes, device=device)

In [ ]:
# model for datasetCoords.py
model_pupil = models.resnet18(pretrained=False)
model_pupil.fc = nn.Linear(in_features=512, out_features=2, dtype=torch.float32)
# model_pupil = PupilDetectModel()

model_pupil

In [ ]:
dataset_pupil = PupilCoreDatasetPupil(
    "datasets/PupilCoreDataset/video5_eye0_video.avi",
    'datasets/PupilCoreDataset/video5_eye0_pupildata.csv',
    "datasets/PupilCoreDataset/video5_eye1_video.avi",
    'datasets/PupilCoreDataset/video5_eye1_pupildata.csv'
)
indices = torch.randperm(len(dataset_pupil)).tolist()
train_part = int(0.8 * len(dataset_pupil))
train_dataset = torch.utils.data.Subset(dataset_pupil, indices[:train_part])
test_dataset = torch.utils.data.Subset(dataset_pupil, indices[train_part:])
dataset_sizes = {
    'train': len(train_dataset),
    'test': len(test_dataset)
}

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)
dataloaders = {
    "train": train_dataloader,
    "test": test_dataloader
}

In [ ]:
def train_second_model(
    model,
    criterion,
    optimizer,
    scheduler,
    num_epochs,
    device,
    dataloaders,
    dataset_sizes,
):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    model = model.to(device)
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs - 1}")
        print("-" * 10)

        # Each epoch has a training and validation phase
        for phase in ["train", "test"]:
            if phase == "train":
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.01
            running_corrects = 0
            losses = []

            # Iterate over data.
            for i, (inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    # print(outputs, labels)
                    outputs.to(device)
                    
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                losses.append(loss.item())
                # print(running_loss)

                if i % 500 == 0:
                    print(f"{phase}, Batch: {i}/{len(dataloaders[phase])} Loss: {np.mean(losses):.4f}")
            if phase == "train":
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            # epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            
            # deep copy the model
            # if phase == "test" and epoch_acc > best_acc:
            #     best_acc = epoch_acc
            #     best_model_wts = copy.deepcopy(model.state_dict())
    return model

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model_pupil.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model_pupil = train_second_model(model_pupil, criterion, optimizer, exp_lr_scheduler, num_epochs=5, dataloaders=dataloaders, dataset_sizes=dataset_sizes, device=device)

In [ ]:
def visualize(model, dataloader, device, num_images: int = 5):
    model.eval()
    fig = plt.figure()
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            
            center_out = list(map(int, outputs[0].cpu().numpy()))
            center_labels = list(map(int, labels[0].cpu().numpy()))
            print(f"center_out: {center_out}", f"center_label: {center_labels}")
            image = np.transpose(inputs[0].cpu().numpy(), (1, 2, 0)).copy()
            
            # fig.add_subplot(1, num_images, i+1)
            cv2.circle(image, center_out, 1, (255, 0, 0), 1)
            cv2.circle(image, center_labels, 1, (0, 255, 0), 1)
            plt.imshow(image)
            plt.show()
            #cv2.imshow("xd", image)
            if i >= num_images:
                
                return

                
# visualize_pupil(model_pupil, dataloaders['test'], device, 5)

In [ ]:
def evaluate(model, dataloaders, device, dataset_sizes):
    model.eval()
    correct = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders["test"]):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            # print(outputs, labels)
            diff = (labels[0][0] - outputs[0][0], labels[0][1] - outputs[0][1])
            # print(diff)
            dist = torch.sqrt(torch.pow(diff[0], 2) + torch.pow(diff[0], 2)) 
            if dist < 2: 
                correct += 1
        
        acc = correct / dataset_sizes['test']
        print(acc)


In [ ]:
evaluate(model_pupil, dataloaders, device, dataset_sizes)

In [ ]:
dataset_corneal = PupilCoreDatasetCornealReflection(
    "datasets/PupilCoreDataset/video5_eye0_video.avi",
    'datasets/PupilCoreDataset/video5_eye0_pupildata.csv',
    "datasets/PupilCoreDataset/video5_eye1_video.avi",
    'datasets/PupilCoreDataset/video5_eye1_pupildata.csv'
)
indices = torch.randperm(len(dataset_corneal)).tolist()
train_part = int(0.8 * len(dataset_corneal))
train_dataset = torch.utils.data.Subset(dataset_corneal, indices[:train_part])
test_dataset = torch.utils.data.Subset(dataset_corneal, indices[train_part:])
dataset_sizes = {
    'train': len(train_dataset),
    'test': len(test_dataset)
}

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)
dataloaders = {
    "train": train_dataloader,
    "test": test_dataloader
}

In [ ]:
#TODO corneal reflection detec


model_corneal = models.resnet18(pretrained=False)
model_corneal.fc = nn.Linear(in_features=512, out_features=2, dtype=torch.float32)

criterion = nn.MSELoss()
optimizer = optim.SGD(model_corneal.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model_corneal = train_second_model(model_corneal, criterion, optimizer, exp_lr_scheduler, num_epochs=2, dataloaders=dataloaders, dataset_sizes=dataset_sizes, device=device)

In [ ]:
evaluate(model_corneal, dataloaders, device, dataset_sizes)

In [ ]:
visualize(model_corneal, dataloaders["test"] ,device, 50)

In [ ]:
#TODO 3 NN system